In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 0 #当日は0、前日は1・・・
date2 = "08/19"
pmc = "42005006" # 店舗ID
SPREADSHEET_KEY = '1nG5qrIY-nrGnwtw-susF4or1fEfhZL6R7HLyDeM5kGs' # スプレッドシートのID
pstart = 1001 # パチンコの1台目
sstart = 2001 # スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+789@gmail.com"
user_pw = "paruparu23"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\Chrome.exe'
chrome_service = ChromeService(executable_path='C:\\Users\\garden026\\WD\\114.0.5735.90\\chromedriver.exe')

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value
s10row = worksheet.cell(11,  30).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
s5table = []
s10table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "125":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "125":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "625":
                    s5table.append(li)
                if unit == "1000":
                    s10table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "625":
                        s5table.append(li)
                    if unit == "1000":
                        s10table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})
wb.values_append("'S10RAW'!F" + str(int(s10row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s10table})

['08/19', '025895', '1001', '46410.0', '-26180.0', '11', '4']
['08/19', '025895', '1002', '30769.0', '43033.0', '49', '7']
['08/19', '025895', '1003', '34359.0', '20765.0', '36', '6']
['08/19', '025895', '1004', '26923.0', '-22569.0', '2', '2']
['08/19', '025895', '1005', '32051.0', '904.0', '21', '6']
['08/19', '025895', '1006', '32564.0', '-25879.0', '4', '4']
['08/19', '025895', '1007', '32821.0', '-15647.0', '12', '5']
['08/19', '025895', '1008', '33846.0', '-7522.0', '15', '4']
['08/19', '025895', '1009', '36154.0', '-5115.0', '21', '7']
['08/19', '025895', '1010', '28462.0', '39723.0', '43', '5']
['08/19', '025895', '1011', '30513.0', '-11133.0', '13', '5']
['08/19', '025895', '1012', '34359.0', '6019.0', '26', '9']
['08/19', '025895', '1013', '37179.0', '5418.0', '27', '4']
['08/19', '025895', '1014', '32308.0', '42431.0', '48', '5']
['08/19', '025895', '1015', '43077.0', '-7522.0', '23', '8']
['08/19', '025895', '1016', '38718.0', '-17754.0', '13', '4']
['08/19', '025895', '101

['08/19', '025897', '1139', '25641.0', '-14143.0', '10', '8']
['08/19', '025817', '1140', '7179.0', '904.0', '20', '6']
['08/19', '025790', '1141', '17949.0', '-11735.0', '10', '6']
['08/19', '025885', '1142', '1795.0', '2107.0', '16', '1']
['08/19', '025884', '1143', '3333.0', '-2707.0', '0', '0']
['08/19', '025887', '1144', '11026.0', '-7221.0', '17', '16']
['08/19', '025647', '1145', '4615.0', '-2707.0', '5', '3']
['08/19', '025764', '1146', '2051.0', '-1805.0', '0', '0']
['08/19', '025892', '1147', '6154.0', '4816.0', '15', '3']
['08/19', '025892', '1148', '2821.0', '2709.0', '9', '2']
['08/19', '025771', '1149', '11026.0', '603.0', '35', '5']
['08/19', '025881', '1150', '3333.0', '-3008.0', '0', '0']
['08/19', '025874', '1151', '7436.0', '-3008.0', '8', '4']
['08/19', '025645', '1152', '10513.0', '1806.0', '7', '2']
['08/19', '025590', '1153', '6410.0', '-3309.0', '2', '2']
['08/19', '025859', '1154', '20769.0', '3612.0', '26', '2']
['08/19', '025779', '1155', '30513.0', '5117.0',

['08/19', '120129', '2086', '4261', '-1991.0', '29', '19', '--']
['08/19', '120129', '2087', '1249', '-1019.0', '5', '6', '--']
['08/19', '120129', '2088', '4040', '3519.0', '66', '19', '--']
['08/19', '120129', '2089', '2471', '-2269.0', '11', '14', '--']
['08/19', '120129', '2090', '955', '-139.0', '8', '6', '--']
['08/19', '120129', '2091', '3134', '3426.0', '58', '19', '--']
['08/19', '120129', '2092', '3099', '4769.0', '72', '11', '--']
['08/19', '120129', '2093', '3618', '185.0', '38', '18', '--']
['08/19', '120129', '2094', '4713', '-1528.0', '30', '20', '--']
['08/19', '120129', '2095', '1584', '1065.0', '21', '8', '--']
['08/19', '120129', '2096', '977', '-509.0', '7', '4', '--']
['08/19', '120129', '2097', '4729', '3056.0', '66', '27', '--']
['08/19', '120129', '2098', '3990', '-509.0', '38', '21', '--']
['08/19', '120144', '2099', '1120', '278.0', '6', '11', '--']
['08/19', '120144', '2100', '643', '1806.0', '4', '14', '--']
['08/19', '120144', '2101', '756', '-231.0', '4', 

['08/19', '119864', '2217', '884', '-231.0', '5', '2', '--']
['08/19', '119864', '2218', '2647', '-833.0', '11', '6', '--']
['08/19', '119864', '2219', '2625', '-694.0', '11', '5', '--']
['08/19', '119864', '2220', '714', '-648.0', '1', '0', '--']
['08/19', '119864', '2221', '5416', '2685.0', '37', '29', '--']
['08/19', '120150', '2222', '4858', '-1065.0', '73', '24', '--']
['08/19', '120150', '2223', '3541', '1759.0', '99', '36', '--']
['08/19', '120150', '2224', '4125', '-2269.0', '56', '21', '--']
['08/19', '120150', '2225', '4212', '2778.0', '99', '41', '--']
['08/19', '120140', '2226', '82', '0', '0', '0', '--']
['08/19', '120140', '2227', '2511', '-1481.0', '55', '0', '--']
['08/19', '120140', '2228', '1859', '-2407.0', '22', '0', '--']
['08/19', '120140', '2229', '1224', '-1389.0', '11', '0', '--']
['08/19', '120140', '2230', '1192', '-1574.0', '11', '0', '--']
['08/19', '120140', '2231', '439', '231.0', '11', '0', '--']
['08/19', '120140', '2232', '746', '-787.0', '11', '0', '-

['08/19', '119959', '3102', '1752', '-1528.0', '11', '6', '--']
['08/19', '120117', '3103', '2382', '741.0', '69', '8', '--']
['08/19', '120117', '3104', '1212', '1574.0', '61', '11', '--']
['08/19', '120117', '3105', '927', '185.0', '30', '9', '--']
['08/19', '120080', '3106', '605', '278.0', '11', '10', '--']
['08/19', '120080', '3107', '596', '93.0', '7', '8', '--']
['08/19', '120080', '3108', '3130', '-1019.0', '34', '35', '--']
['08/19', '119995', '3109', '4923', '417.0', '22', '39', '--']
['08/19', '119995', '3110', '2557', '-417.0', '12', '18', '--']
['08/19', '119995', '3111', '6894', '5833.0', '30', '75', '--']
['08/19', '119995', '3112', '1562', '-1389.0', '3', '5', '--']
['08/19', '119945', '3113', '1723', '-694.0', '2', '4', '--']
['08/19', '119945', '3114', '5883', '2083.0', '21', '8', '--']
['08/19', '119945', '3115', '2380', '-1065.0', '4', '5', '--']
['08/19', '119945', '3116', '956', '-556.0', '1', '2', '--']
['08/19', '120015', '3117', '6833', '2130.0', '95', '43', '-

['08/19', '120118', '4008', '1594', '-787.0', '3', '2', '--']
['08/19', '120069', '4009', '1244', '278.0', '19', '4', '--']
['08/19', '120069', '4010', '1115', '-370.0', '11', '3', '--']
['08/19', '120121', '4011', '1910', '139.0', '53', '4', '--']
['08/19', '120121', '4012', '1416', '833.0', '45', '4', '--']
['08/19', '120139', '4013', '1475', '1806.0', '27', '6', '--']
['08/19', '120139', '4014', '2010', '-1250.0', '11', '4', '--']
['08/19', '120073', '4015', '4394', '231.0', '50', '23', '--']
['08/19', '120094', '4016', '9', '0', '0', '0', '--']
['08/19', '120015', '4017', '1887', '1296.0', '30', '10', '--']
['08/19', '120015', '4018', '5490', '-2037.0', '47', '25', '--']
['08/19', '120120', '4019', '3617', '-2361.0', '20', '0', '--']
['08/19', '120120', '4020', '3222', '2361.0', '40', '0', '--']
['08/19', '120123', '4021', '3432', '-3333.0', '18', '7', '--']
['08/19', '119984', '4022', '2677', '417.0', '13', '6', '--']
['08/19', '120115', '4023', '1112', '-972.0', '9', '1', '--']
[

['08/19', '119945', '4139', '4339', '-1296.0', '8', '12', '--']
['08/19', '119945', '4140', '4746', '-463.0', '5', '12', '--']
['08/19', '119945', '4141', '5995', '-139.0', '17', '16', '--']
['08/19', '119995', '4142', '3374', '-231.0', '18', '29', '--']
['08/19', '119995', '4143', '5574', '1019.0', '31', '55', '--']
['08/19', '119995', '4144', '2973', '-972.0', '15', '21', '--']
['08/19', '119995', '4145', '5167', '-2870.0', '14', '27', '--']
['08/19', '119995', '4146', '1806', '-833.0', '9', '10', '--']
['08/19', '120083', '4147', '1586', '-880.0', '6', '6', '--']
['08/19', '120095', '4148', '463', '-417.0', '7', '4', '--']
['08/19', '120095', '4149', '1171', '-139.0', '15', '11', '--']
['08/19', '120053', '4150', '5527', '-1019.0', '99', '21', '--']
['08/19', '120053', '4151', '4729', '833.0', '99', '19', '--']
['08/19', '120053', '4152', '4413', '2269.0', '99', '18', '--']
['08/19', '120053', '4153', '5373', '-509.0', '99', '25', '--']
['08/19', '120084', '4154', '2954', '694.0', '

{'spreadsheetId': '1nG5qrIY-nrGnwtw-susF4or1fEfhZL6R7HLyDeM5kGs',
 'updates': {'spreadsheetId': '1nG5qrIY-nrGnwtw-susF4or1fEfhZL6R7HLyDeM5kGs',
  'updatedRange': 'S10RAW!F7058:M7105',
  'updatedRows': 48,
  'updatedColumns': 8,
  'updatedCells': 384}}